In [1]:
import sys
import json
import numpy as np
import pandas as pd

sys.path.append("..")

from graphstructure.simple_substructure import SubstructureSimple
from graphstructure.metrics import MetricFunction, REAL_LIFE_LOADS

{'NONE': 0, 'H': 1, 'Z': 2, 'IZ': 3, 'ZH': 4, 'IZH': 5, 'K': 6, 'X': 7, 'XH': 8}
Dropped brace5 because of 0 var
Dropped layer_height4 because of 0 var
FAMD detected categorical columns Index(['legs', 'n_layers', 'brace0', 'brace1', 'brace2', 'brace3', 'brace4'], dtype='object')
FAMD detected continuous columns Index(['total_height', 'radius_bottom', 'radius_top', 'layer_height0',
       'layer_height1', 'layer_height2', 'layer_height3'],
      dtype='object')
FAMD detected categorical columns Index(['legs', 'n_layers', 'brace0', 'brace1', 'brace2', 'brace3', 'brace4'], dtype='object')
FAMD detected continuous columns Index(['total_height', 'radius_bottom', 'radius_top', 'layer_height0',
       'layer_height1', 'layer_height2', 'layer_height3'],
      dtype='object')
MCA PROCESS ELIMINATED 5  COLUMNS SINCE THEIR MISS_RATES >= 99%


/home/panagiotou/Desktop/Kallikrates/venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [2]:
metrics = MetricFunction.from_loads(REAL_LIFE_LOADS)

Running with objectives: ['Cost', 'wave', 'combined', 'compression', 'tipover', 'torque']


In [3]:
import math
def test_all(designs):
    objs = [] 
    for d in designs:
        sub = SubstructureSimple.from_dict(d)
        obj_values = np.array(metrics(sub)[0])
        if any([math.isinf(x) for x in obj_values]):
            print('This design cant be tested skipping:', d)
            continue
            # print('This design cant be tested in ops and thus got scores of 0.0:', d)
            # obj_values[np.where(obj_values == 0.0)] = np.inf
        objs.append(obj_values)
    return objs

In [4]:
def to_df(objs):
    # print(metrics.metric_names)
    return pd.DataFrame(objs, columns=metrics.metric_names)

## Load real structures

In [5]:
real = []

with open("data/real_structures.json") as f:
    real = json.load(f)

synthetic = []
synthetic_name = "data/dgmm(real)_100000.json"

with open(synthetic_name) as f:
    synthetic = json.load(f)

In [6]:
import os
def evaluate(designs, file_path):
    objs = test_all(designs)
    df = to_df(objs)
    data_merged = [{**d, **df.loc[i]} for i, d in enumerate(designs)]
    filename = os.path.splitext(os.path.basename(file_path))[0]
    output_filename = "data/evaluated/" + filename +"_evaluated.json"
    with open(output_filename, "w") as json_file:
        json.dump(data_merged, json_file, indent=4)


In [7]:
evaluate(synthetic, synthetic_name)

WARNING BandSPDLinLapackSolver::solve() -factorization failed, matrix singular U(i,i) = 0, i= 181
WARNING Linear::solveCurrentStep() -the LinearSOE failed in solve()
StaticAnalysis::analyze() - the Algorithm failed at step: 0 with domain at load factor 1
OpenSees > analyze failed, returned: -3 error flag
WARNING BandSPDLinLapackSolver::solve() -factorization failed, matrix singular U(i,i) = 0, i= 139
WARNING Linear::solveCurrentStep() -the LinearSOE failed in solve()
StaticAnalysis::analyze() - the Algorithm failed at step: 0 with domain at load factor 1
OpenSees > analyze failed, returned: -3 error flag


This design cant be tested skipping: {'legs': 4, 'total_height': 49.707689071846175, 'connection_types': ['X', 'X', 'X', 'X'], 'radius_bottom': 12.381256018187154, 'radius_top': 7.591941771748563, 'n_layers': 5, 'layer_heights': [11.250962899235754, 27.385499360957976, 34.07044966113509], 'id': 5179}


In [8]:
asdfas

NameError: name 'asdfas' is not defined

In [ ]:
# for des in synthetic:
#     SubstructureSimple.from_dict(des).plot(rotation=30)

In [ ]:
# SubstructureSimple.from_dict(synthetic[-1]).plot(rotation=30)

In [ ]:
objs_synthetic = test_all(synthetic)
df_synthetic = to_df(objs_synthetic)
df_synthetic

In [ ]:
output_filename = "synthetic.json"
synthetic_merged = [{**d, **df_synthetic.loc[i]} for i, d in enumerate(synthetic)]
with open(output_filename, "w") as json_file:
    json.dump(synthetic_merged, json_file, indent=4)

In [ ]:
import plotly.express as px
df_real['source'] = "real"
df_synthetic['source'] = "generated"
df_real['source_n'] = 1
df_synthetic['source_n'] = 2
df_mixed = pd.concat([
    df_real, df_synthetic
])
df_mixed.index = list(range(len(df_mixed)))
fig = px.parallel_coordinates(df_mixed, color="source_n", color_continuous_scale=px.colors.diverging.Tealrose)
fig.show()

## Load generated structures

In [ ]:
designs = []
with open("bigrunnew.json") as f:
    designs = json.load(f)
    
objs_gen = test_all(designs)
df_gen = to_df(objs_gen)

In [ ]:
print(df_gen.shape)
df_designs = pd.DataFrame(designs)
print(df_gen.max())
both = pd.concat([df_designs, df_gen], axis = 1)

with open("optimal_collection_2023-01-17_10-27-57.json", "w") as f:
    json.dump(both.to_json(), f)

In [ ]:
fig = px.parallel_coordinates(df_gen)
fig.show()

## Cluster the generated data

In [ ]:
from graphstructure.app_interface import cluster_means

objs_gen = np.array(objs_gen)
objs_gen = objs_gen[np.isfinite(objs_gen).all(axis=1)]
cluster_centers = cluster_means(objs_gen, n=10)
df_gen_clusters = to_df(cluster_centers)

## Compare real vs generated

In [ ]:
styles = {"Titles": {'legs': 'Number of legs', 'total_height': 'Total height', 'radius_bottom': 'Radius bottom', 'radius_top': 'Radius top', 'n_layers': 'Number of layers', 'brace0': 'Brace type (first layer)', 'brace1': 'Brace type (second layer)', 'brace2': 'Brace type (third layer)', 'brace3': 'Brace type (forth layer)', 'brace4': 'Brace type (fifth layer)', 'layer_height0': 'First layer height', 'layer_height1': 'Second layer height', 'layer_height2': 'Third layer height', 'layer_height3': 'Forth layer height'},
            "Objectives": {'Cost': 'Cost', 'plausibility': 'Plausibility', 'wave': 'Wave test', 'compression': 'Compression test', 'tipover': 'Pushover test', 'torque': 'Tortion test', 'combined': 'Combined test'},
            "Colors": {'Cost': 'rgb(31, 119, 180)', 'plausibility': 'rgb(255, 127, 14)', 'wave': 'rgb(44, 160, 44)', 'compression': 'rgb(214, 39, 40)', 'tipover': 'rgb(148, 103, 189)', 'torque': 'rgb(140, 86, 75)', 'real': 'rgba(255, 180, 180, 0.1)', 'generated': 'rgba(200, 200, 200, 0.1)', 'generated_clusters': 'rgba(0, 0, 0, 1.0)'},
            "Thickness": {'real': 1, 'generated': 1, 'generated_clusters': 4}}
# THESE ARE CLOSER TO THE COLORS IN THE STATISTICS PLOT
#styles["Colors"] |= {'real': 'rgb(90, 190, 240)', 'generated': 'rgb(255, 200, 90)', 'generated_clusters': 'rgb(140, 60, 0)'}

In [ ]:
import plotly.graph_objects as go

# comment away whatever you dont want to see in the plot
# entries with
# key: name of trace
# value: dict(plot_order, df, color) (colors are set after this)
sources = {
    "real":               {"plot_order": 1, "df": df_real}, 
    "generated":          {"plot_order": 0, "df": df_gen},
    "generated_clusters": {"plot_order": 2, "df": df_gen_clusters},
}

# add color for the traces
for k, v in sources.items():
    v["color"] = styles["Colors"][k]

for source, v in sources.items():
    v["df"]["source"] = source
    v["df"]["source_n"] = v["plot_order"]

df_mixed = pd.concat([x["df"] for _, x in sources.items()])
df_mixed.index = list(range(len(df_mixed)))

dims = []
obj_order = ["compression", "tipover", "torque", "wave", "combined", "Cost", "plausibility"]
for name in obj_order:
    dct = dict(label=styles["Objectives"][name], values=df_mixed[name])
    dims.append(dct)
    
prios = [x["plot_order"] for x in sources.values()]
min_, max_ = min(prios), max(prios)    

colorscale = []
for source, dct in sorted(sources.items(), key=lambda x: x[1]["plot_order"]):
    colorscale.append([(dct["plot_order"]-min_)/max_, dct["color"]])

fig = go.Figure(data=
    go.Parcoords(
        labelfont = dict(size=16), # Objective names font
        rangefont = dict(size=12), # min max values font
        tickfont = dict(size=10), # interpolated values font
        line = dict(color=df_mixed['source_n'],
                    colorscale=colorscale),
        dimensions = dims
    )
)

fig.show()

In [ ]:
# Show plots of structures closest to the cluster centers

%matplotlib notebook
import numpy as np
obs = np.array(objs_gen)
for centroid in cluster_centers:
    dists = np.linalg.norm(obs - centroid, axis=1)
    idx = np.argmin(dists)
    SubstructureSimple.from_dict(designs[idx]).plot()